In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.corpus import stopwords
import pickle
import nltk
from sklearn.model_selection import KFold,cross_val_score,train_test_split,cross_val_predict
from sklearn import svm
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
df = pd.read_excel('C:/Users/alexa/OneDrive/Documenten/Thesis Lab/dataset_final.xlsx')

In [3]:
df

,Index,Filename,Countrycode,Label,Sentence_translated,Sentence_original
0,0,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,Law on the Establishment of an Economic Stabil...,'Gesetz zur Errichtung eines Wirtschaftsstabil...
1,1,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,I p. 543 (no.,"'I S. 543 (Nr.',"
2,2,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,"14); Valid from March 28th, 2020 4 changes | P...",'14); Geltung ab 28.03.2020 4 Änderungen | Dr...
3,3,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,28th,"'28.',"
4,4,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,March 2020 StFG § 6- § 8- § 10- § 14e- § 15- §...,'März 2020 StFG § 6- § 8- § 10- § 14e- § 15- ...
...,...,...,...,...,...,...
5445,5445,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,Second final provision.,"'Disposición final segunda.',"
5446,5446,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,Entry into force.,"'Entrada en vigor.',"
5447,5447,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,This royal decree-law will enter into force on...,'Este real decreto-ley entrará en vigor el dí...
5448,5448,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,"Given in Madrid- on March 10, 2020.","'Dado en Madrid- el 10 de marzo de 2020.',"


# removal of stopwords (in case of multi-lingual model)

In [4]:
stops = set(stopwords.words('spanish'))
stops_french = set(stopwords.words('french'))
stops_italian = set(stopwords.words('italian'))
stops_german = set(stopwords.words('german'))

stops = stops.union(stops_french, stops_italian, stops_german)

In [5]:
df['Sentence_original'] = df['Sentence_original'].apply(word_tokenize)

In [6]:
df['Sentence_original'] = df['Sentence_original'].apply(lambda words: [word.lower() for word in words if word not in stops])

In [7]:
df['Sentence_original'] = df['Sentence_original'].apply(lambda x: ' '.join(map(str, x)))

In [8]:
df

,Index,Filename,Countrycode,Label,Sentence_translated,Sentence_original
0,0,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,Law on the Establishment of an Economic Stabil...,'gesetz errichtung wirtschaftsstabilisierungsf...
1,1,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,I p. 543 (no.,"' i s. 543 ( nr . ' ,"
2,2,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,"14); Valid from March 28th, 2020 4 changes | P...",'14 ) ; geltung ab 28.03.2020 4 änderungen | d...
3,3,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,28th,"'28 . ' ,"
4,4,011_DEU_2020-03-27c_2022-05-20_10:59:17.4.txt,DEU,NONE,March 2020 StFG § 6- § 8- § 10- § 14e- § 15- §...,'märz 2020 stfg § 6- § 8- § 10- § 14e- § 15- §...
...,...,...,...,...,...,...
5445,5445,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,Second final provision.,"'disposición final segunda . ' ,"
5446,5446,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,Entry into force.,"'entrada vigor . ' ,"
5447,5447,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,This royal decree-law will enter into force on...,'este real decreto-ley entrará vigor día sigui...
5448,5448,001_ES_2020-03-10_2022-05-07_11:57:05.0.txt,ESP,NONE,"Given in Madrid- on March 10, 2020.","'dado madrid- 10 marzo 2020 . ' ,"


# Majority Baseline

In [4]:
len(df['Label'])

5450

In [23]:
df['Label'].value_counts()

NONE                                             4123
FINANCIAL POLICY: DIRECT PAYMENTS                 387
EMPLOYMENT POLICY                                 254
FINANCIAL POLICY: GUARANTEES                      205
FISCAL POLICY                                     188
SOCIAL WELFARE POLICY                              99
FINANCIAL POLICY: RESTRUCTURING OF LOAN TERMS      75
CONSUMER PROTECTION                                48
PRICE CONTROL                                      38
INDUSTRIAL POLICY                                  33
Name: Label, dtype: int64

In [5]:
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(df['Sentence_translated'], df['Label'])

DummyClassifier(strategy='stratified')

In [6]:
dummy_clf.predict(df['Sentence_translated'])

array(['NONE', 'FINANCIAL POLICY: DIRECT PAYMENTS', 'NONE', ..., 'NONE',
       'NONE', 'NONE'], dtype=object)

In [7]:
dummy_clf.score(df['Sentence_translated'], df['Label'])

0.5897247706422019

# Training

In [43]:
X_train, X_test, y_train, y_test = train_test_split(df['Sentence_translated'], df['Label'], test_size=0.2, random_state=111)

In [44]:
from sklearn.utils import compute_class_weight 

In [45]:
class_weights = compute_class_weight(
                                        class_weight = 'balanced',
                                        classes = np.unique(y_train),
                                        y = y_train                                                    
                                    )

class_weights = dict(zip(np.unique(y_train), class_weights))

class_weights

{'CONSUMER PROTECTION': 12.11111111111111,
 'EMPLOYMENT POLICY': 2.2244897959183674,
 'FINANCIAL POLICY: DIRECT PAYMENTS': 1.384126984126984,
 'FINANCIAL POLICY: GUARANTEES': 2.759493670886076,
 'FINANCIAL POLICY: RESTRUCTURING OF LOAN TERMS': 6.8125,
 'FISCAL POLICY': 2.945945945945946,
 'INDUSTRIAL POLICY': 16.14814814814815,
 'NONE': 0.13188142770719904,
 'PRICE CONTROL': 13.212121212121213,
 'SOCIAL WELFARE POLICY': 5.662337662337662}

In [46]:
vectorizer = FeatureUnion([
    ('tfidf_word', TfidfVectorizer(ngram_range=(1, 2), strip_accents='unicode', lowercase=True, stop_words='english')),
    ('tfidf_word2', TfidfVectorizer(analyzer='word', ngram_range=(1, 5), strip_accents='unicode', lowercase=True, stop_words='english')),
    ('tfidf_char', TfidfVectorizer(analyzer='char', ngram_range=(1, 10), strip_accents='unicode', lowercase=True, stop_words='english'))
])

parameter: class_weight='balanced'

In [47]:
clf = Pipeline([
('vectorize', vectorizer),
('classify',LinearSVC(class_weight = 'balanced'))
])

In [48]:
tfidf_bow_translated = clf.fit(X_train,y_train)

In [26]:
pickle.dump(tfidf_bow_translated, open('tfidf_bow_translated_weighted.pkl', 'wb'))

In [49]:
test_prediction_1 = tfidf_bow_translated.predict(X_test)

In [50]:
print("prediction on trained_model_1 (word trigram + char bi-gram)")
print(classification_report(y_test, test_prediction_1))

prediction on trained_model_1 (word trigram + char bi-gram)
                                               precision    recall  f1-score   support

                          CONSUMER PROTECTION       0.67      0.33      0.44        12
                            EMPLOYMENT POLICY       0.73      0.79      0.76        58
            FINANCIAL POLICY: DIRECT PAYMENTS       0.78      0.82      0.80        72
                 FINANCIAL POLICY: GUARANTEES       0.80      0.83      0.81        47
FINANCIAL POLICY: RESTRUCTURING OF LOAN TERMS       0.80      0.73      0.76        11
                                FISCAL POLICY       0.80      0.88      0.83        40
                            INDUSTRIAL POLICY       0.50      0.50      0.50         6
                                         NONE       0.93      0.93      0.93       817
                                PRICE CONTROL       0.80      0.80      0.80         5
                        SOCIAL WELFARE POLICY       0.73      0.36   

# Cross Validation on tfidfvectorizer

#1 Using English-translated Sentences

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df['Sentence_translated'], df['Label'], test_size=0.2, random_state=0)

In [23]:
from sklearn import metrics
scores_f1 = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')

In [24]:
print(scores_f1)


[0.86009174 0.89220183 0.91055046 0.89449541 0.88073394 0.86926606
 0.88073394 0.8646789  0.8853211  0.91743119]


In [25]:
import statistics
average = statistics.mean(scores_f1)
print(average)

0.8855504587155963


#2 Using Multilingual Sentences

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df['Sentence_original'], df['Label'], test_size=0.2, random_state=0)

In [305]:
from sklearn import metrics
scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='precision_macro')

In [306]:
print(scores)

[0.69789183 0.82553114 0.78979432 0.75129831 0.7538534 ]


In [307]:
import statistics
average = statistics.mean(scores)
print(average)

0.7636737996441066


# Classification report combined with Cross Validation

In [28]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer

def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    return accuracy_score(y_true, y_pred) # return accuracy score

In [30]:
# Nested CV with parameter optimization
nested_score = cross_val_score(clf, X_train, y_train, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))

print(nested_score )

C:\Users\alexa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\alexa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\alexa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                               precision    recall  f1-score   support

                          CONSUMER PROTECTION       1.00      0.25      0.40         4
                            EMPLOYMENT POLICY       0.65      0.71      0.68        21
            FINANCIAL POLICY: DIRECT PAYMENTS       0.70      0.81      0.75        32
                 FINANCIAL POLICY: GUARANTEES       0.71      0.62      0.67        16
FINANCIAL POLICY: RESTRUCTURING OF LOAN TERMS       0.75      0.50      0.60         6
                                FISCAL POLICY       0.79      0.69      0.73        16
                            INDUSTRIAL POLICY       0.00      0.00      0.00         3
                                         NONE       0.91      0.92      0.91       329
                                PRICE CONTROL       0.33      0.50      0.40         2
                        SOCIAL WELFARE POLICY       0.75      0.86      0.80         7

                                     accu

                                               precision    recall  f1-score   support

                          CONSUMER PROTECTION       0.67      0.50      0.57         4
                            EMPLOYMENT POLICY       0.67      0.82      0.73        22
            FINANCIAL POLICY: DIRECT PAYMENTS       0.58      0.66      0.62        32
                 FINANCIAL POLICY: GUARANTEES       0.76      0.81      0.79        16
FINANCIAL POLICY: RESTRUCTURING OF LOAN TERMS       0.67      0.67      0.67         6
                                FISCAL POLICY       0.93      0.93      0.93        15
                            INDUSTRIAL POLICY       1.00      0.50      0.67         2
                                         NONE       0.93      0.91      0.92       328
                                PRICE CONTROL       0.00      0.00      0.00         3
                        SOCIAL WELFARE POLICY       0.75      0.75      0.75         8

                                     accu